<a href="https://colab.research.google.com/github/mohanrajmit/AgeCalculation/blob/master/Agecalculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Age Estimation for Indian Movie Face Dataset like Young, Old and Middle

In [4]:
!git clone https://github.com/mohanrajmit/AgeCalculation.git

fatal: destination path 'AgeCalculation' already exists and is not an empty directory.


In [2]:
# keras imports

from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.models import model_from_json
from keras.layers import Input
from keras.applications.inception_resnet_v2 import InceptionResNetV2

# other imports
from sklearn.preprocessing import LabelEncoder
import numpy as np
import glob
import cv2
import h5py
import os
import json
import datetime
import time
import pandas as pd




Using TensorFlow backend.


In [0]:
data=pd.read_csv("AgeCalculation/dataset/train.csv")

image_name = data["ID"]
label_name = data["Class"]

features = []
labels = []

In [6]:
print(data.head())

          ID   Class
0    377.jpg  MIDDLE
1  17814.jpg   YOUNG
2  21283.jpg  MIDDLE
3  16496.jpg   YOUNG
4   4487.jpg  MIDDLE


In [15]:
base_model = Xception(weights="imagenet")
model = Model(input=base_model.input, output=base_model.get_layer('avg_pool').output)
image_size = (299,299)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("av...)`
  


In [0]:
dataset_path="AgeCalculation/dataset/Train/"

In [17]:
for image_name1,label in zip(image_name,label_name):
	
	print("the image name  {} and label{}" .format(image_name1,label))
	image_path= str(dataset_path+image_name1)
	#print(image_path)
	#image =cv2.imread(image_path)
	#images.append(image)
	#labels.append(label)

	img = image.load_img(image_path, target_size=image_size)
	x = image.img_to_array(img)
	x = np.expand_dims(x, axis=0)
	x = preprocess_input(x)
	feature = model.predict(x)
	flat = feature.flatten()
	features.append(flat)
	labels.append(label)

the image name  377.jpg and labelMIDDLE
the image name  17814.jpg and labelYOUNG
the image name  21283.jpg and labelMIDDLE
the image name  16496.jpg and labelYOUNG
the image name  4487.jpg and labelMIDDLE
the image name  6283.jpg and labelMIDDLE
the image name  23495.jpg and labelYOUNG
the image name  7100.jpg and labelYOUNG
the image name  6028.jpg and labelYOUNG
the image name  22617.jpg and labelOLD
the image name  11177.jpg and labelYOUNG
the image name  2462.jpg and labelMIDDLE
the image name  24116.jpg and labelMIDDLE
the image name  17689.jpg and labelMIDDLE
the image name  728.jpg and labelMIDDLE
the image name  3003.jpg and labelMIDDLE
the image name  14408.jpg and labelOLD
the image name  6679.jpg and labelYOUNG
the image name  15152.jpg and labelOLD
the image name  24784.jpg and labelMIDDLE
the image name  9970.jpg and labelYOUNG
the image name  22550.jpg and labelOLD
the image name  150.jpg and labelYOUNG
the image name  7379.jpg and labelMIDDLE
the image name  15387.jpg an

In [18]:
# encode the labels using LabelEncoder
le = LabelEncoder()
le_labels = le.fit_transform(labels)

# get the shape of training labels
print ("[STATUS] training labels: {}".format(le_labels))
print ("[STATUS] training labels shape: {}".format(le_labels.shape))

# save features and labels
h5f_data = h5py.File("features.h5", 'w')
h5f_data.create_dataset('dataset_1', data=np.array(features))

h5f_label = h5py.File("label.h5", 'w')
h5f_label.create_dataset('dataset_1', data=np.array(le_labels))

h5f_data.close()
h5f_label.close()

[STATUS] training labels: [0 2 0 ... 0 0 0]
[STATUS] training labels shape: (19906,)


In [80]:
list(le.inverse_transform([0, 1, 2]))

['MIDDLE', 'OLD', 'YOUNG']

In [0]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import numpy as np
import h5py
import os
import json
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

In [0]:
from __future__ import print_function

In [21]:
# import features and labels
h5f_data  = h5py.File("features.h5", 'r')
h5f_label = h5py.File("label.h5", 'r')

features_string = h5f_data['dataset_1']
labels_string   = h5f_label['dataset_1']

features = np.array(features_string)
labels   = np.array(labels_string)

h5f_data.close()
h5f_label.close()

# verify the shape of features and labels
print ("[INFO] features shape: {}".format(features.shape))
print ("[INFO] labels shape: {}".format(labels.shape))

print ("[INFO] training started...")
# split the training and testing data

[INFO] features shape: (19906, 2048)
[INFO] labels shape: (19906,)
[INFO] training started...


In [22]:
(trainData, testData, trainLabels, testLabels) = train_test_split(np.array(features),
                                                                  np.array(labels),
                                                                  test_size=0.10,
                                                                  random_state=9)

print("[INFO] splitted train and test data...")
print("[INFO] train data  : {}".format(trainData.shape))
print("[INFO] test data   : {}".format(testData.shape))
print("[INFO] train labels: {}".format(trainLabels.shape))
print("[INFO] test labels : {}".format(testLabels.shape))



[INFO] splitted train and test data...
[INFO] train data  : (17915, 2048)
[INFO] test data   : (1991, 2048)
[INFO] train labels: (17915,)
[INFO] test labels : (1991,)


Logisitic Regression Algorithm to train the pretrained model extracted features.

In [26]:
# use logistic regression as the model
print("[INFO] creating model...")
model_ml = LogisticRegression(random_state=9)
model_ml.fit(trainData, trainLabels)

print("[INFO] evaluating model...")
f = open("results.txt", "w")

	
# evaluate the model of test data
preds = model_ml.predict(testData)

# write the classification report to file
f.write("{}\n".format(classification_report(testLabels, preds)))
f.close()

classifier_path="classifier.pickle"

# dump classifier to file
print("[INFO] saving model...")
pickle.dump(model_ml, open(classifier_path, 'wb'))



[INFO] creating model...


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[INFO] evaluating model...
[INFO] saving model...


In [0]:
!rm -r AgeCalculation

In [92]:
!git clone https://github.com/mohanrajmit/AgeCalculation.git

Cloning into 'AgeCalculation'...
remote: Enumerating objects: 26096, done.
remote: Counting objects: 100% (26096/26096), done.
remote: Compressing objects: 100% (18690/18690), done.
remote: Total 26096 (delta 7408), reused 26087 (delta 7404), pack-reused 0
Receiving objects: 100% (26096/26096), 57.07 MiB | 29.88 MiB/s, done.
Resolving deltas: 100% (7408/7408), done.
Checking out files: 100% (26546/26546), done.


In [0]:
test_dataset="AgeCalculation/dataset/Test/"
#!pip install imutils

In [0]:
data_test=pd.read_csv("AgeCalculation/dataset/test.csv")

image_name_test = data_test["ID"]

features = []
labels_test = []

In [98]:
print(len(image_name_test))

6636


In [99]:
base_model = Xception(weights="imagenet")
model = Model(input=base_model.input, output=base_model.get_layer('avg_pool').output)
image_size = (299,299)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("av...)`
  


In [102]:
images_label =['MIDDLE', 'OLD', 'YOUNG']
images_name = []
labels_test = []
for image_name1 in image_name_test:
  
  image_pa = test_dataset+image_name1
  print(image_pa)
  
  img = image.load_img(image_pa, target_size=image_size)
  #image_list = image_name1.split("/")[-1]
  print(image_pa)
  #make = imagePath.split("/")[-2]
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  feature = model.predict(x)
  flat = feature.flatten()
  flat = flat.reshape(1, -1)
  
  preds = model_ml.predict(flat)
  
 
  preds= images_label[int(preds)]
  print(preds)
  labels_test.append(preds)
  images_name.append(image_name1)

AgeCalculation/dataset/Test/25321.jpg
AgeCalculation/dataset/Test/25321.jpg
MIDDLE
AgeCalculation/dataset/Test/989.jpg
AgeCalculation/dataset/Test/989.jpg
YOUNG
AgeCalculation/dataset/Test/19277.jpg
AgeCalculation/dataset/Test/19277.jpg
OLD
AgeCalculation/dataset/Test/13093.jpg
AgeCalculation/dataset/Test/13093.jpg
MIDDLE
AgeCalculation/dataset/Test/5367.jpg
AgeCalculation/dataset/Test/5367.jpg
MIDDLE
AgeCalculation/dataset/Test/19851.jpg
AgeCalculation/dataset/Test/19851.jpg
MIDDLE
AgeCalculation/dataset/Test/10384.jpg
AgeCalculation/dataset/Test/10384.jpg
YOUNG
AgeCalculation/dataset/Test/24567.jpg
AgeCalculation/dataset/Test/24567.jpg
YOUNG
AgeCalculation/dataset/Test/2029.jpg
AgeCalculation/dataset/Test/2029.jpg
MIDDLE
AgeCalculation/dataset/Test/4956.jpg
AgeCalculation/dataset/Test/4956.jpg
MIDDLE
AgeCalculation/dataset/Test/26131.jpg
AgeCalculation/dataset/Test/26131.jpg
MIDDLE
AgeCalculation/dataset/Test/17407.jpg
AgeCalculation/dataset/Test/17407.jpg
MIDDLE
AgeCalculation/datas

In [103]:
print(np.array(images_name).shape)
print(np.array(labels_test).shape)

(6636,)
(6636,)


In [0]:
sub = pd.DataFrame({'Class': labels_test, 'ID': images_name})
sub.to_csv('output.csv', index=False)